## Packages

From termainal I did:

```
pipenv install
pipenv install jupyter pymysql sqlalchemy requests
```


In [30]:
import pymysql.cursors
import requests
from datetime import datetime
from sqlalchemy import create_engine
import hashlib 
import os
import json
import time
from collections import defaultdict
from flask import jsonify
import re
import pandas as pd
from gspread_pandas import Spread, Client

# variables we'll need
host = os.environ['DBHOST']
port = 3306
dbname = "collaborate"
user = os.environ['DBUSER']
password = os.environ['DBPASSWORD']

In [2]:
#### Main function ####

def handler(incoming):
    
    ## Put code here
    
    
    return True

In [3]:
########

In [4]:
# create sqlalchemy engine
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                   .format(user=user,
                           pw=password,
                           host=host,
                           db=dbname))

In [5]:
# this will become a loop of projects pulled from a 
# master google spreadsheet
project = "propublicafruittest-pyrata"
spreadsheet_id = "1uHvY_Z0lpGdvAgkfTt-sNXMe5yGOiq0NZJju0aLej1E"

In [6]:
# Get the data we need
dbConnection    = engine.connect()

df = pd.read_sql(f"SELECT `identifier`, `item_key`, `item_value` FROM `data_pieces` WHERE `project` = '{project}'", dbConnection)
columns_df = pd.read_sql(f"SELECT * FROM `column_tracker` WHERE `project` = '{project}'", dbConnection)
text_df = pd.read_sql(f"SELECT * FROM `text_log` WHERE `project` = '{project}'", dbConnection)
first_contact_df = pd.read_sql(f"SELECT * FROM `first_contact` WHERE `project` = '{project}'", dbConnection)

dbConnection.close()

In [7]:
pivoted = df.pivot(index='identifier', columns='item_key', values=['item_value'])

In [8]:
# pivoted

,item_value
item_key,Default Welcome Intent
identifier,
test-session-7e9945fa1a0a,Hola


In [9]:
pivoted.columns = pivoted.columns.get_level_values(1)
pivoted.reset_index(inplace=True) 

In [11]:
# pivoted

In [12]:
# pivoted.columns

In [13]:
columns_df.sort_values(by=['created_at'], inplace=True)

In [14]:
ordered_columns = columns_df['col'].tolist()

In [15]:
ordered_columns.insert(0, "identifier")

In [16]:
# ordered_columns

In [17]:
pivoted_ordered = pivoted[ordered_columns]

In [18]:
# pivoted_ordered

In [19]:
# text_df

In [20]:
text_concat = text_df.groupby(['identifier'])['raw_text'].apply(' | '.join).reset_index()



In [21]:
merge1 = pd.merge(pivoted_ordered, text_concat, on="identifier")

In [22]:
# merge1

In [23]:
first_contact_df.sort_values(by=['created_at'], inplace=True)

In [24]:
final_table = pd.merge(first_contact_df, merge1, on="identifier")

In [26]:
final_table

,proj_ident_hash,project,identifier,created_at,Default Welcome Intent,raw_text
0,dd8b594c008ad32f17f60f6a857f9e9e,propublicafruittest-pyrata,test-session-7e9945fa1a0a,2020-07-14 11:46:36,Hola,Hola


In [28]:
# drop the project columns for the spreadsheet
final_table.drop(columns=['proj_ident_hash', 'project'], inplace = True)

In [29]:
# final_table

,identifier,created_at,Default Welcome Intent,raw_text
0,test-session-7e9945fa1a0a,2020-07-14 11:46:36,Hola,Hola


## Trying a service account

In [32]:
spread = Spread(spreadsheet_id)

In [33]:
spread.df_to_sheet(final_table, index=False, sheet='Sheet1', start='A1', replace=True)